In [1]:
import os
import cv2
import ipywidgets.widgets as widgets
import threading
import time
from IPython.display import display
import numpy as np

# 设置摄像头显示组件
image_widget = widgets.Image(format='jpeg', width=640, height=480)

# bgr8转jpeg格式
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

# 打开摄像头
cap = cv2.VideoCapture(0)

# 检查摄像头是否成功打开
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# 设置图像宽度和高度
width = 640
height = 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# 设置工作目录为项目根目录
os.chdir(r'/home/asus/Project/')

# 加载预训练的SSD模型
modelFile = "res10_300x300_ssd_iter_140000.caffemodel"
configFile = "deploy.prototxt"

# 检查文件是否存在
if not os.path.exists(modelFile):
    print(f"Error: {modelFile} does not exist.")
else:
    print(f"{modelFile} exists.")

if not os.path.exists(configFile):
    print(f"Error: {configFile} does not exist.")
else:
    print(f"{configFile} exists.")

try:
    net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
except Exception as e:
    print(f"Failed to load network: {e}")
    exit()

def camera():
    try:
        display(image_widget)  # 显示摄像头组件
        while True:
            ret, frame = cap.read()
            if not ret:
                print("无法读取帧")
                break

            # 获取图像的高度和宽度
            h, w = frame.shape[:2]

            # 创建一个blob对象
            blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

            # 设置输入blob到网络
            net.setInput(blob)

            # 进行前向传播以获取预测结果
            detections = net.forward()

            # 遍历检测结果
            for i in range(detections.shape[2]):
                confidence = detections[0, 0, i, 2]

                # 过滤掉低置信度的结果
                if confidence > 0.5:
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")

                    # 绘制矩形框标记人脸
                    cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)

            # 更新图像组件
            image_widget.value = bgr8_to_jpeg(frame)
            time.sleep(0.1)
    except KeyboardInterrupt:
        print("程序关闭！")
        cap.release()

# 创建并启动线程
camera_thread = threading.Thread(target=camera)
camera_thread.start()

res10_300x300_ssd_iter_140000.caffemodel exists.
deploy.prototxt exists.


Image(value=b'', format='jpeg', height='480', width='640')